In [ ]:
import logging
import sys
import time
import osqp
import numpy as np
import scipy as sp

from scipy import signal
from scipy import sparse
from threading import Event

dt = 0.002


params = dict()
params['g'] = 9.81
params['m'] = 0.027

g = params['g']
m = params['m']

Ixx = 2.3951E-5
Iyy = 2.3951E-5
Izz = 3.2347E-5

# Construct the state space system for the quadrotor (based on https://arxiv.org/pdf/1908.07401)
A = np.zeros([12,12])
A[0:3,3:6] = np.eye(3)
A[3:6,6:9] = np.array([[0, -g, 0], 
                        [g, 0, 0],
                        [0, 0, 0 ]])
A[6:9,9:12] = np.eye(3)

B = np.zeros([12,4])
#B[4,0] = -1/m
B[5,0] = -1/m
B[9:12, 1:4] = np.array([[1/Ixx, 0, 0], 
                            [0, 1/Iyy, 0],
                            [0, 0, 1/Izz]])

C = np.zeros([6,12])
C[0:3,0:3] = np.eye(3)
C[3:6, 6:9] = np.eye(3)

D = np.zeros([6,4])

sys = signal.StateSpace(A, B, C, D)
discrete_sys = sys.to_discrete(dt)


formatted_A = np.array2string(discrete_sys.A, separator=',', threshold=np.inf, max_line_width=np.inf)
formatted_A = formatted_A.replace('[', '').replace(']', '')
print("A:")
print("----------------------")
print(formatted_A)

formatted_B = np.array2string(discrete_sys.B, separator=',', threshold=np.inf, max_line_width=np.inf)
formatted_B = formatted_B.replace('[', '').replace(']', '')
print("B:")
print("----------------------")
print(formatted_B)


A:
----------------------
 1.000e+00, 0.000e+00, 0.000e+00, 1.000e-02, 0.000e+00, 0.000e+00, 0.000e+00,-4.905e-04, 0.000e+00, 0.000e+00,-1.635e-06, 0.000e+00,
  0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 1.000e-02, 0.000e+00, 4.905e-04, 0.000e+00, 0.000e+00, 1.635e-06, 0.000e+00, 0.000e+00,
  0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 1.000e-02, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
  0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,-9.810e-02, 0.000e+00, 0.000e+00,-4.905e-04, 0.000e+00,
  0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 9.810e-02, 0.000e+00, 0.000e+00, 4.905e-04, 0.000e+00, 0.000e+00,
  0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
  0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 1.000e-02, 0.000e+00, 0.000e+00,
  0.000e+00, 0.000e+00, 0.000e+00, 0.